# Setup

In [8]:
import datetime

import tensorflow as tf
from tensorflow.keras import (
    datasets,
    layers,
    Sequential,
    callbacks,
    losses, 
    optimizers,
    metrics
)

In [2]:
%load_ext tensorboard

# Load the Data

In [3]:
mnist = datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(10, activation='softmax')
    ])

# Using TensorBoard with Keras Model.fit()

In [4]:
model = create_model()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x = x_train,
          y = y_train,
          validation_data = [x_test, y_test],
          epochs=20,
          callbacks = [tensorboard_cb])

Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Epoch 1/20


2022-03-11 16:32:50.003399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-11 16:32:50.003519: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-03-11 16:32:50.155278: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  10/1875 [..............................] - ETA: 10s - loss: 1.8370 - accuracy: 0.4281 

2022-03-11 16:32:50.314582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1871/1875 [============================>.] - ETA: 0s - loss: 0.2184 - accuracy: 0.9345

2022-03-11 16:32:59.875619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 11s 6ms/step - loss: 0.2181 - accuracy: 0.9346 - val_loss: 0.1095 - val_accuracy: 0.9656
Epoch 2/20
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0966 - accuracy: 0.9704 - val_loss: 0.0762 - val_accuracy: 0.9760
Epoch 3/20
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0670 - accuracy: 0.9787 - val_loss: 0.0789 - val_accuracy: 0.9753
Epoch 4/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0517 - accuracy: 0.9836 - val_loss: 0.0680 - val_accuracy: 0.9799
Epoch 5/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0411 - accuracy: 0.9868 - val_loss: 0.0621 - val_accuracy: 0.9811
Epoch 6/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0322 - accuracy: 0.9895 - val_loss: 0.0630 - val_accuracy: 0.9808
Epoch 7/20
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0285 - accuracy: 0.9904 - val_loss: 0.0649 - val_accuracy: 0.9

In [ ]:
%tensorboard --logdir logs/fit

# Using Tensorboard with other methods

In [7]:
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

train_dataset = train_dataset.shuffle(60000).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [10]:
# Create the objects to compile the model
loss_function = losses.SparseCategoricalCrossentropy()
optimizer = optimizers.Adam()

train_loss = metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = metrics.SparseCategoricalAccuracy('test_accuracy')

In [11]:
# Define the training and test functions

def train_step(model, optimizer, x_train, y_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_function(y_train, predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y_train, predictions)

def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_function(y_test, predictions)
    
    test_loss(loss)
    test_accuracy(y_test, predictions
                  )

In [12]:
current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [15]:
model = create_model()

EPOCHS = 20

for epoch in range(EPOCHS):
    for (x_train, y_train) in train_dataset:
        train_step(model, optimizer, x_train, y_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_loss.result(), step=epoch)
    
    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_loss.result(), step=epoch)
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    
    # Reset metrics every epoch
    for metric in [train_loss, train_accuracy, test_loss, test_accuracy]:
        metric.reset_states()

Epoch 1, Loss: 0.1786699742078781, Accuracy: 94.57725524902344, Test Loss: 0.09580734372138977, Test Accuracy: 96.99000549316406
Epoch 2, Loss: 0.08433691412210464, Accuracy: 97.45833587646484, Test Loss: 0.08396423608064651, Test Accuracy: 97.36000061035156
Epoch 3, Loss: 0.05963600426912308, Accuracy: 98.18500518798828, Test Loss: 0.06449270993471146, Test Accuracy: 98.01000213623047
Epoch 4, Loss: 0.04526938498020172, Accuracy: 98.58333587646484, Test Loss: 0.060837168246507645, Test Accuracy: 98.20000457763672
Epoch 5, Loss: 0.03566374257206917, Accuracy: 98.89500427246094, Test Loss: 0.055467940866947174, Test Accuracy: 98.22000122070312
Epoch 6, Loss: 0.02797708474099636, Accuracy: 99.11333465576172, Test Loss: 0.06324158608913422, Test Accuracy: 98.01000213623047
Epoch 7, Loss: 0.02371121384203434, Accuracy: 99.22333526611328, Test Loss: 0.060003191232681274, Test Accuracy: 98.22000122070312
Epoch 8, Loss: 0.019709572196006775, Accuracy: 99.33333587646484, Test Loss: 0.056200679

In [16]:
%tensorboard --logdir logs/gradient_tape